# Training and Testing an MDRNN on Different Models

This script tests the MDRNN on different trained models and generates some output plots.

In [ ]:
%matplotlib inline
import random
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from context import *
import empi_mdrnn

def generate_test_performance(weights_file, net, num_steps=1500):
    """Generate num_steps of unconstrained data from a network (net), with weights_file's weights"""
    net.model.load_weights(weights_file)
    net.prepare_model_for_running()
    sample = empi_mdrnn.random_sample()
    p = net.generate_performance(sample, num_steps)
    perf_df = pd.DataFrame({'t':p.T[0], 'x':p.T[1]})
    perf_df['time'] = perf_df.t.cumsum()
    return perf_df
    
def plot_nn_output(output_df, name="unknown", colour='b'):
    """Plot the data in output_df assuming it has columns x and time"""
    ax = perf_df.plot(x='time', y='x', kind="line", color=colour, figsize=(15,4), legend=False)
    ax.set_xlabel("seconds")
    ax.set_ylabel("position")
    ax.get_figure().savefig('../images/model_output/'+name+'_model_output.pdf', dpi=300, bbox_inches="tight")
    ax.get_figure().savefig('../images/model_output/'+name+'_model_output.png', dpi=300, bbox_inches="tight")

### Setup network and test

Setup network and test with each trained model. Main parameters to change here are `mdrnn_units` and `num_steps`

In [ ]:
# Model Hyperparameters
mdrnn_units = 32
mdrnn_mixes = 5
mdrnn_layers = 2
# Sampling parameters
pitemp = 1
sigmatemp = 0.01
# Number of steps to sample
num_steps = 500

empi_mdrnn.MODEL_DIR = "./models/"
net = empi_mdrnn.PredictiveMusicMDRNN(mode=empi_mdrnn.NET_MODE_RUN,
                                              dimension=2,
                                              n_hidden_units=mdrnn_units,
                                              n_mixtures=mdrnn_mixes,
                                              layers=mdrnn_layers)
net.pi_temp = pitemp
net.sigma_temp = sigmatemp

# test noise model.
model_file = "../models/musicMDRNN-dim2-layers2-units"+str(mdrnn_units)+"-mixtures5-scale10-noise.h5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate slightly nicer plot.
plot_nn_output(perf_df, name="noise"+str(mdrnn_units))

# Load and test synthetic model.
model_file = "../models/musicMDRNN-dim2-layers2-units"+str(mdrnn_units)+"-mixtures5-scale10-synth.h5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate slightly nicer plot.
plot_nn_output(perf_df, name="synth"+str(mdrnn_units))

# Load and test synthetic model.
model_file = "../models/musicMDRNN-dim2-layers2-units"+str(mdrnn_units)+"-mixtures5-scale10-human.h5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate slightly nicer plot.
plot_nn_output(perf_df, name="human"+str(mdrnn_units))

In [ ]:
# ## Investigate Output
# window = 100
# for n in [1000,2000,3000,4000,5000,6000]:
#     print("Window:", str(n),'to',str(n+window))
#     plt.plot(perf_df[n:n+window].time, perf_df[n:n+window].x, '.r-')
#     plt.show()